Amazon Nova Canvas offers a task type called "IMAGE_VARIATION". Image variation is a versatile capability with a range of uses. Depending on the parameters you provide, image variation can help you...

Produce new images of a specific subject based on images of that subject
Guide the model to produce images in a particular visual style by providing examples of that style
Mix and combine styles from multple reference images to create unique styles
This opens up new creative possibilities for generating sophisticated and engaging images without the need for complex prompt engineering or model fine-tuning.

## Use case
OctankFashion can leverage the Image Variation feature to create unique designs. For example, they can start with a reference image of a wedding dress design generate many different variations with subtle or drastic changes, blending the original design with the stylistic elements from the reference images. This allows them to iterate on concepts more rapidly, exploring different silhouettes, textures, and color palettes without the time and cost of physical prototyping.

In [1]:
import io
import json
import base64
import boto3
from botocore.config import Config
from PIL import Image
from utils import save_image, plot_images, plot_images_for_comparison

bedrock_runtime_client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1",
    config=Config(
        read_timeout=5 * 60
    ),  # IMPORTANT: Increase the read timeout to 5 minutes to support longer operations.
)
image_generation_model_id = "amazon.nova-canvas-v1:0"
output_dir = "output"

In [2]:
referenced_image= "data/wedding_dress.png"
similarity_strength=( 0.7 )
text = "improve the facial expression include the face of the subject"
#outpainting_mode = "PRECISE"  # Either "DEFAULT" or "PRECISE"

seed = 97


In [3]:
with open(referenced_image,"rb") as image_file:
    reference_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")
body = json.dumps(
        {
            "taskType": "IMAGE_VARIATION",
            "imageVariationParams": {"text": text,
                                #"negativeText":negative_text,
                                "images":[reference_image_base64],
                                "similarityStrength": similarity_strength,
                                #"outPaintingMode": outpainting_mode,
                                },
            "imageGenerationConfig": {
                "numberOfImages": 1,  # Number of images to generate, up to 5
                "width": 1024,
                "height": 1024,
                "cfgScale": 6.5,  # How closely the prompt will be followed
                "seed": seed,  # Any number from 0 through 858,993,459
                "quality": "premium",  # Quality of either "standard" or "premium"
            },
        }
    )
response = bedrock_runtime_client.invoke_model(
        body=body,
        modelId=image_generation_model_id,
        accept="application/json",
        contentType="application/json",
    )

response_body = json.loads(response.get("body").read())

base64_images = response_body.get("images")
image_path = f"{output_dir}/03-image-variation-{seed}.png"
save_image(base64_images[0], image_path)

print(f"Saved to {image_path}")
generated_img = [
        Image.open(io.BytesIO(base64.b64decode(base64_image)))
        for base64_image in base64_images
    ]
#generated_images.append(generated_img[0])


Saved to output/03-image-variation-97.png
